In [2]:
import warnings
warnings.filterwarnings('ignore')

from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import regex as re
import torch
from farsi_tools import stop_words
from hazm import word_tokenize, Lemmatizer
from pymongo import MongoClient
from sentence_transformers import util
from symspellpy import SymSpell, Verbosity
from tqdm import tqdm
from const.constance import *


lemmatizer = Lemmatizer()


# STOP WORD

In [3]:
with open(stopword_path, encoding="utf8") as f:
    stop = f.readlines()
# cleaning stopwords
stop_word = [word.replace('\n', '') for word in stop]
stop_word = [re.sub('[\\u200c]', ' ', word) for word in stop_word]
stop_word.extend(stop_words())


# mongo db

In [4]:
def context_mongo(collection_name):
    client = MongoClient(host=host, port=port)
    client_my = client['nlp']
    my_coll = client_my[collection_name]
    return my_coll

p# ==ol

In [5]:
def load_glove_model(glove_file):
    print("loading glove model")
    model = KeyedVectors.load_word2vec_format(glove_file, binary=False)
    print(f"loaded glove model , {len(model)}")
    return model

In [6]:
model = load_glove_model(glove_path)

loading glove model
loaded glove model , 240548


# spell checker

In [7]:
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=8)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

# word 2 vector sy

In [8]:
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_cbow_path, binary=True)

In [9]:
test = "تعیین مقادیر پیش فرض در ماژول جلسات"
test1 = "تعیین مقادیر پیش فرض در افزونه جلسات"
text = "ویدئو انیمیشن برای همکارات بفرست"

In [10]:
word2vec_model.most_similar_cosmul('IT',topn=300)


[('ERP', 0.8689835667610168),
 ('اتوماسیون', 0.8578999042510986),
 ('راهبری', 0.8481571078300476),
 ('ICT', 0.8443616628646851),
 ('کسب\u200cوکار', 0.8366156220436096),
 ('زیرساختی', 0.8302525877952576),
 ('Management', 0.8295701146125793),
 ('SAP', 0.8287033438682556),
 ('خودکارسازی', 0.827669084072113),
 ('مدیریتی', 0.8246448636054993),
 ('Service', 0.8239153623580933),
 ('Security', 0.8237847089767456),
 ('مشارکتی', 0.8226858973503113),
 ('سیسکو', 0.8220700621604919),
 ('توزیع\u200cشده', 0.8212320804595947),
 ('لجستیک', 0.8208287358283997),
 ('یکپارچه\u200cسازی', 0.8204160928726196),
 ('PLC', 0.8197720646858215),
 ('زیرساخت', 0.8197266459465027),
 ('توسعه\u200cای', 0.8187692761421204),
 ('زیربنایی', 0.8179969787597656),
 ('مشاوره\u200cای', 0.8171239495277405),
 ('نرم\u200cافزاری', 0.816767156124115),
 ('مدیریت', 0.8163818717002869),
 ('IETF', 0.8143932819366455),
 ('ATM', 0.8135349154472351),
 ('Systems', 0.8115798234939575),
 ('CRM', 0.8107935786247253),
 ('سخت\u200cافزارها', 0.808

# Pre

In [11]:
def read_dataset_csv(csv_fine_path):
    df = pd.read_csv(csv_fine_path)  # read csv file
    data_list = df.to_dict(orient='records')  # reshape to dict
    return data_list

In [12]:
def encode_to_db(collection_name):
    collection_name = MongoClient(host, port)
    client = collection_name['nlp']
    my_collection = client['encoded_new_coll_collection']
    main_collection = client['new_coll']
    dict_list = [i['job'] for i in main_collection.find({}, {'job'})]
    for i in tqdm(dict_list):
        if my_collection.find_one({'text': i}):
            pass
        else:
            my_collection.insert_one({
                'text': i,
                'encoded': word_embedding_method(i)
            })


In [68]:

def word_embedding_method(sentence):
    try:
        encoded_word_list = []
        for i in preprocess(sentence):
            if i in word2vec_model:
                encoded_word_list.append(word2vec_model[i])
            else:
                continue
        if encoded_word_list is None:
            return None
        else:
            return np.mean(encoded_word_list, axis=0).tolist()
    except KeyError as e:
        return None

In [69]:
word_embedding_method('محتوای شبکه اجتماعی لینکدین')

[1.215616226196289,
 0.4286821186542511,
 -0.5207251906394958,
 0.4606791138648987,
 1.3914198875427246,
 1.1369383335113525,
 -0.5813952684402466,
 0.5204800963401794,
 0.47460970282554626,
 -1.4391437768936157,
 0.4674311578273773,
 -0.14379426836967468,
 -0.8068406581878662,
 0.41642752289772034,
 1.3531839847564697,
 -0.33421286940574646,
 0.5531848669052124,
 0.05917990207672119,
 0.14695502817630768,
 1.0609160661697388,
 0.40442919731140137,
 0.3077820837497711,
 -0.008428089320659637,
 0.9650437831878662,
 -0.20326948165893555,
 1.0492972135543823,
 0.7273272275924683,
 -0.6738163232803345,
 -0.45814332365989685,
 1.5417709350585938,
 0.5649698376655579,
 -1.565555214881897,
 -0.005331411957740784,
 -0.4879113733768463,
 -0.3059764802455902,
 0.8167935609817505,
 0.3207032382488251,
 0.5578347444534302,
 0.12139920890331268,
 0.9566700458526611,
 -0.9906855225563049,
 0.3096960186958313,
 -1.165270209312439,
 -0.978934645652771,
 0.11888518929481506,
 -0.6433796286582947,
 -0.3

# preprocess

In [70]:

def preprocess(raw_text):
    words = word_tokenize(raw_text)  # split a sentence to words and return an array
    words = [i for i in words if i not in stop_word]
    spell_checker_list = []
    for i in range(len(words)):  # iterate in word array for checking spell
        if not sym_spell.lookup(words[i], Verbosity.ALL, max_edit_distance=3):  # if word not exists ignore the word
            continue
        else:
            word_matched = sym_spell.lookup(words[i], Verbosity.ALL, max_edit_distance=3)
            for i in word_matched[:1]:  # take first similar word to our incorrect word
                spell_checker_list.append(i)
    #lemmatize = [lemmatizer.lemmatize(wo
    # rd) for word in words]

    split_lemm_words = []
    for i in words:

        if "#" not in i:
            split_lemm_words.append(i)
        else:
            split_lemm_words.extend(i.split("#"))
    clean_words = list(set([w for w in split_lemm_words if w not in stop_word]))  # remove some word like "و ,با, ..."

    return clean_words



# word synonyms 

In [71]:
def calculate_similarity_of_words(sentence1, sentence2):
    sentence_1 = preprocess(sentence1)

    sentence_word_similarity = {}
    for i in sentence_1:
        get_similarity = word2vec_model.most_similar_cosmul(i, topn=20)
        lemm = [lemmatizer.lemmatize(word[0]) for word in get_similarity]
        cleaning_words = [re.sub(r'[\u200c]', '', word) for word in lemm]
        sentence_word_similarity[i] = list(set(cleaning_words))
    gf_list = {}
    for sent in sentence2:
        sentence_2 = preprocess(sent)
        reformed_sentence = sentence_2.copy()
        for i, j in sentence_word_similarity.items():
            for x in sentence_2:
                if x in j:
                    reformed_sentence = [i if word == x else word for word in reformed_sentence]
        gf_list = {**gf_list, **{sent: ' '.join(reformed_sentence)}}
    return gf_list


# *Similarity*

In [72]:

def final_similarity(sentence):
    my_collection = context_mongo('new_version_of_cleaned_dataset')

    vector_1 = np.mean([word_embedding_method(sentence)], axis=0)
    result_pass = [calculate_similarity_of_words(sentence, [i['cleaned_text'] for i in my_collection.find()][:50])]

    op = {}
    for i in result_pass:
        for j in i.values():
            if len(j.split(' ')) > 1:
                op[j] = np.mean([word_embedding_method(j)], axis=0)
            else:
                if type(word_embedding_method(j)) == float:
                    continue
                else:
                    op[j] = word_embedding_method(j)
    result_dict = {}
    result = util.cos_sim(vector_1, np.array([j if i else None for i, j in op.items()], dtype=float))[0]
    top_results = torch.topk(result, k=5)
    for score, idx in zip(top_results[0], top_results[1]):
        result_dict[list(result_pass[0].keys())[int(idx)]] = np.round(score.numpy() * 100, 2)
    return result_dict

In [73]:
final_similarity("محتوای شبکه اجتماعی لینکدین")

{'محتوای شبکه اجتماعی لینکدین': 100.0,
 'محتوای شبکه اجتماعی فیس بوک': 92.69,
 'تهیه و نگارش استراتژی اولیه': 87.98,
 'ایده برای روش های بدست آوردن دیتا قابل اتکا': 80.94,
 'محتوای صفحات وب مدیریت پروژه': 69.35}



# Sent By Sent

In [79]:
def calculate_similarity_of_words(sentence1, sentence2):
    sentence_1 = preprocess(sentence1)
    sentence_word_similarity = {}
    for i in sentence_1:
        get_similarity = model.most_similar_cosmul(i, topn=20)
        lemm = [lemmatizer.lemmatize(word[0]) for word in get_similarity]
        cleaning_words = [re.sub(r'[\u200c]', '', word) for word in lemm]
        sentence_word_similarity[i] = list(set(cleaning_words))

    sentence_2 = preprocess(sentence2)

    reformed_sentence = sentence_2.copy()
    for i, j in sentence_word_similarity.items():
        for x in sentence_2:
            if x in j:
                reformed_sentence = [i if word == x else word for word in reformed_sentence]
    gf_list = ' '.join(reformed_sentence)
    return gf_list


def single_sim(sentence1, sentence2):
    vector_1 = np.mean([word_embedding_method(sentence1)], axis=0)
    result_pass = calculate_similarity_of_words(sentence1, sentence2)

    op = {}
    if len(result_pass.split(' ')) > 1:
        op[result_pass] = np.mean([word_embedding_method(result_pass)], axis=0)
    else:
        if type(word_embedding_method(result_pass)) == float:
            pass
        else:
            op[result_pass] = word_embedding_method(result_pass)
    result_dict = {}
    result = util.cos_sim(vector_1, np.array([j if i else None for i, j in op.items()], dtype=float))[0]
    return result



In [80]:
text = "محتوای شبکه اجتماعی لینکدین"
text1 = "محتوای شبکه اجتماعی فبسبوک"
single_sim(text, text1)

tensor([0.9988], dtype=torch.float64)

# for faster calculation

In [81]:
my_collection = context_mongo('new_version_of_cleaned_dataset')
sentences = [i['cleaned_text'] for i in my_collection.find()]

In [103]:
def find_upload(sentences):
    all_predicted_sentences = {}
    for i in tqdm(sentences):
        tokenized = preprocess(i)
        if len(tokenized) > 0:
            ref_with_sim = {}
            for word in tokenized:
                sim_words_array = []
                try:
                    get_similarity = word2vec_model.most_similar_cosmul(word, topn=30)
                    for similarity in get_similarity:
                        sim_words_array.append(similarity[0])
                except:
                    pass
                ref_with_sim = {**ref_with_sim, **{word: sim_words_array}}
            maked_sentence = []
            for tokenized_word in preprocess(i):
                for l, m in ref_with_sim.items():
                    if l == tokenized_word:
                        for ih in m:
                            maked_sentence.append(str(i).replace(l, ih))
            all_predicted_sentences = {**all_predicted_sentences, **{i: maked_sentence}}
        else:
            continue
    return all_predicted_sentences


In [104]:
all_syn = context_mongo('hahahauiofferaiguogipouydgdfgfdgfgfdgyoh')

In [105]:
for i, j in find_upload(sentences[:100]).items():
    if all_syn.find_one({'name': i}):
        pass
    else:
        all_syn.insert_one({
            "name": i,
            "encoded": j
        })

100%|██████████| 100/100 [01:13<00:00,  1.35it/s]


In [106]:
syn_encoded = context_mongo('222hatyyiofyiuiouiouiytiha')

In [108]:

for i in tqdm(all_syn.find({}, {"_id": False})):
    if syn_encoded.find_one({'text': i['name']}):
        pass
    else:
        syn_encoded_list = []
        for m in i['encoded']:
            syn_encoded_list.append(np.mean([word_embedding_method(m)], axis=0).tolist())
        if syn_encoded_list:
            syn_encoded.insert_one({
                'text': i['name'],
                'generated_sent': i['encoded'],
                'mean_encoded': syn_encoded_list

            })


100it [50:49, 30.50s/it]


In [138]:
def result(ref):
    vector_1 = np.mean([word_embedding_method(ref)], axis=0)
    res = {}
    for i in tqdm(syn_encoded.find({}, {"_id": False})):
        result = util.cos_sim(vector_1.tolist(), [ c for c in i['mean_encoded'] if type(c) !=float ])
        res[i['text']]=np.mean(sorted(result[0].detach().numpy(),reverse=True)[:20])
        #res[i['text']] = result[0].detach().numpy()
    return list(sorted(res.items(), key=lambda item: item[1], reverse=True))[:20]

In [139]:

result('مارکتینگ email')


100it [00:00, 169.91it/s]


[('معماری ایمیل مارکتینگ', 0.49875498),
 ('محتوای ایمیل مارکتینگ خوش آمدگویی', 0.4974696),
 ('برنامه ریزی برای ایمیل مارکتینگ', 0.49633646),
 ('محتوای ایمیل مارکتینگ', 0.4960641),
 ('محتوای ایمیل مارکتینگ روز اول', 0.4960641),
 ('محتوای ایمیل مارکتینگ روز سوم', 0.4960641),
 ('محتوای ایمیل مارکتینگ روز هغتم', 0.4960641),
 ('محتوای ایمیل مارکتینگ روز', 0.4960641),
 ('the business model canvas', 0.4869298),
 ('بازنگری تارگت و پرسونا هم تیک', 0.44960827),
 ('محتوای ایمیل مارکتینگ روز چهاردهم', 0.42841488),
 ('تهیه منبع ایمیل ها از تریدر و تحلیلگر ها', 0.42230058),
 ('راهنمای مدیریت پروژه', 0.40749127),
 ('دیجیتال مارکتینگ و توسعه بازار', 0.40657917),
 ('طراحی اینفوگرام معماری ایمیل مارکتینگ', 0.40590554),
 ('طراحی فرمت کلی', 0.3970964),
 ('امکانات تایم شیت', 0.38673624),
 ('محتوای صفحات وب مدیریت پروژه', 0.3802409),
 ('لینک جستجوی پیشرفته در زیر فیلتر های صفحه kp', 0.37133622),
 ('محتوای شبکه اجتماعی فیس بوک', 0.3660497)]

In [130]:
\from nltk import ngrams
from collections import defaultdict


class to_ngram:
    def __init__(self, data_set, n_gram: list):
        super(to_ngram, self).__init__()
        self.n_gram = n_gram
        self.data_set = data_set

    def __calculate_imbalanced_label(self):
        job_count = self.data_set.who.value_counts()
        job_count = job_count[job_count < 50]
        return job_count.index

    def __do_not_know(self):
        job_count = self.__calculate_imbalanced_label()
        output_series = pd.Series()
        for i in job_count:
            tasks = self.data_set[self.data_set['who'] == i]['job']
            list_of_task = []
            for task in tasks:
                for num in self.n_gram:
                    word = task.split()
                    word = ngrams(word, num)
                    result = [' '.join(ngram) for ngram in word]
                    if len(result) > 0:
                        list_of_task.append(result)
            output_series.loc[i] = list_of_task
        return output_series

    def __sep_dataset(self):
        output_series = self.__do_not_know()
        #full_list=defaultdict(list)
        final_dict = {}
        for who, jobs in output_series.items():
            for y in jobs:
                for z in y:
                    final_dict[z] = who
        return final_dict

    def __result(self):
        final_dict = self.__sep_dataset()
        dataframe = pd.DataFrame([final_dict]).transpose().reset_index()
        dataframe.rename(columns={'index': "job", int(0): "who"}, inplace=True)
        return dataframe

    def __call__(self):
        return self.__result()


SyntaxError: unexpected character after line continuation character (3758142603.py, line 1)

In [95]:
dataset=context_mongo('all_syn')
li=[]
for i in dataset.find({},{'_id':False})[23:24]:
    li.append(i['name'])

In [96]:
class n_gram:
    def __init__(self,data_set,ngram:list):
        super(n_gram,self).__init__()
        self.data_set=data_set
        self.ngram=ngram
        
    def do_not_know(self):
        list_of_task = []
        for i in self.data_set:
            word=i.split()
            word=ngrams(word,2)
            
            result = [' '.join(ngram) for ngram in word]
            if len(result) > 0:
                list_of_task.append(result)
        return list_of_task

In [97]:
nnn=n_gram(li,[2,3])

In [103]:
for i in nnn.do_not_know():
    print(i)

['طراحی اینفوگرام', 'اینفوگرام معماری', 'معماری ایمیل', 'ایمیل مارکتینگ']


In [52]:
تغییر منوی سمت راست

بیزینس ناظر کار


SyntaxError: invalid syntax (1652023506.py, line 1)

In [52]:
import pandas as pd

In [64]:
dataset=pd.read_csv('../datasets/tasks_report (17).csv')

In [65]:
dataset=dataset[dataset['نام پروژه'].str.contains('واحد تولید')]['عنوان کار']

In [71]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [72]:
dataset.reset_index(drop=True,inplace=True)

In [73]:
dataset

0            ویرایش اطلاعات کاربران از طریق ایمپورت فایل
1      اصلاحات مربوط به عدم نمایش فیلد کارشناس در ثبت...
2      ارسال پیامک با توجه به داکیومنت راهنمای پیوست شده
3      اصلاح لایسنس به نحویکه به mac address وابسته ن...
4                                          کارهای متفرقه
                             ...                        
909      نمایش کلمه undefined  در تعیین برچسب دستور جلسه
910                                   آپدیت زیر دامنه ها
911       ایجاد دکمه برای بازگرداندن وضعیت به حالت اولیه
912    در پنل مدیریت چارت سازمانی از ماژول‌های پایه ب...
913                                   تغیییر بعضی فریزها
Name: عنوان کار, Length: 914, dtype: object

In [74]:
dataset.to_csv('task.csv', index=False, encoding='utf-8')

In [78]:
c=pd.read_csv('../datasets/task.csv')

In [79]:
c[['عنوان کار']]

,عنوان کار
0,ویرایش اطلاعات کاربران از طریق ایمپورت فایل
1,اصلاحات مربوط به عدم نمایش فیلد کارشناس در ثبت...
2,ارسال پیامک با توجه به داکیومنت راهنمای پیوست شده
3,اصلاح لایسنس به نحویکه به mac address وابسته ن...
4,کارهای متفرقه
...,...
909,نمایش کلمه undefined در تعیین برچسب دستور جلسه
910,آپدیت زیر دامنه ها
911,ایجاد دکمه برای بازگرداندن وضعیت به حالت اولیه
912,در پنل مدیریت چارت سازمانی از ماژول‌های پایه ب...
